**Team members**

1) Gireesh V

2) Sidharth khurana

3) Srikrishna Srinivasan

4) Vipin kumar

In [ ]:
 pip install tweepy==4.5.0

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 66 kB 2.6 MB/s 
     |████████████████████████████████| 62 kB 1.4 MB/s 
  Attempting uninstall: requests
    Found existing installation: requests 2.23.0
    Uninstalling requests-2.23.0:
      Successfully uninstalled requests-2.23.0
  Attempting uninstall: tweepy
    Found existing installation: tweepy 3.10.0
    Uninstalling tweepy-3.10.0:
      Successfully uninstalled tweepy-3.10.0


In [ ]:
pip install pyodbc

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 472 kB 4.8 MB/s 


In [ ]:
pip install sqlalchemy

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import tweepy as tw

In [ ]:
import pandas as pd

In [ ]:
my_api_key = "lQGGJlmt6hBQrTdHO3qv855FT"
my_api_secret = "ERgzsW56jj8DzZkXYTixw0yBLbgEEEdd2PEM9t6jn6fXRMRxUm"
MY_BEARER_TOKEN = "AAAAAAAAAAAAAAAAAAAAAAbRgwEAAAAAZK01IGpPMy7T5fiAPezczxtEaTA%3DjoG8pZOWEEYFcPRggcom1eOsVg68xzcXGxVCxE3pxL1DRxlXv3"

In [ ]:
def cleanse_tweets(tweet):
  new_tweet = ''
  keywords = []
  handles = []
  medias = []

  i=0
  while (i<len(tweet)):
    if (tweet[i]=='#'):
      keyword = ''
      while (True):
        i+=1
        if (i<len(tweet) and (tweet[i].isalnum() or tweet[i]=='_')):
          keyword += tweet[i]
        else: 
          keywords.append(keyword)
          break

    elif (tweet[i]=='@'):
      handle = ''
      while (True):
        new_tweet+=tweet[i]
        i+=1
        if (i<len(tweet) and (tweet[i].isalnum() or tweet[i]=='_')):
          handle += tweet[i]
        else: 
          handles.append(handle)
          break

    elif (tweet[i:i+13]=='https://t.co/'):
      media = 'https://t.co/'
      i+=13
      while (True):
        i+=1
        if (i<len(tweet) and tweet[i].isalnum()):
          media += tweet[i]
        else: 
          medias.append(media)
          break
    else:
      new_tweet+=tweet[i]
      i+=1

  return {'text':new_tweet.strip(),'keywords':','.join(keywords),'handles':','.join(handles),'medias':','.join(medias)}

In [ ]:
def twitter_search(query,start_time,end_time):
    client = tw.Client(bearer_token=MY_BEARER_TOKEN)
     # get tweets from the API
    tweets = client.search_recent_tweets(query=query,
                                         start_time=start_time,
                                         end_time=end_time,
                                         tweet_fields = ["created_at", "text", "source"],
                                         user_fields = ["name", "username", "location", "verified"],
                                         #max_results = 10,
                                         expansions='author_id'
                                         )
    # Creating a dictionary of the tweets
    global tweets_dict
    tweets_dict = {'id':[],'location':[],'name':[],'username':[],'date':[],'tweet':[],'handles':[],'keywords':[],'media':[],'verified' :[]}

    for i in range(len(tweets.data)):
        tweets_dict['id'].append(tweets.data[i]['id'])
        tweets_dict['date'].append(str(tweets.data[0]['created_at'].day) + '/' + str(tweets.data[0]['created_at'].month) + '/' +
                                  str(tweets.data[0]['created_at'].year))
        tweet_data = cleanse_tweets(tweets.data[i]['text'])
        tweets_dict['tweet'].append(tweet_data['text'])
        tweets_dict['handles'].append(tweet_data['handles'])
        tweets_dict['keywords'].append(tweet_data['keywords'])
        tweets_dict['media'].append(tweet_data['medias'])
        for j in range(len(tweets.includes['users'])):
            if tweets.includes['users'][j]['id'] == tweets.data[i]['author_id']:
                tweets_dict['name'].append(tweets.includes['users'][j]['name'])
                tweets_dict['username'].append(tweets.includes['users'][j]['username'])
                tweets_dict['verified'].append(tweets.includes['users'][j]['verified'])
                tweets_dict['location'].append(tweets.includes['users'][j]['location'])
    
    # Printing out the tweets
    i = 0
    while i < len(tweets.data):
        if (i==10):break
        for key in tweets_dict.keys():
            print(key,'-',tweets_dict[key][i])
        print('---------------------------------')
        i += 1   
    # Adding this dictionary to a DataFrame
    global tweets_df
    tweets_df = pd.DataFrame(tweets_dict)
    tweets_df.drop('handles',axis = 1,inplace=True)
    tweets_df.date = pd.to_datetime(tweets_df.date)
    display(tweets_df)


In [ ]:
query = "#lfc lang:en -is:retweet is:verified"
start_time = "2022-10-03T00:00:00Z"
end_time = "2022-10-06T00:00:00Z"

In [ ]:
twitter_search(query,start_time,end_time)

id - 1577793253197242368
location - Liverpool
name - Liverpool FC News
username - LivEchoLFC
date - 5/10/2022
tweet - "When they move away from home, it is time for me to think about myself."
handles - 
keywords - LFC
media - https://t.co/greNJMInP
verified - True
---------------------------------
id - 1577788221693509632
location - Liverpool
name - Liverpool FC News
username - LivEchoLFC
date - 5/10/2022
tweet - Liverpool evening headlines
handles - 
keywords - LFC
media - https://t.co/sHY3Zesvt
verified - True
---------------------------------
id - 1577782825574694912
location - Dublin & London
name - Kevin Palmer
username - RealKevinPalmer
date - 5/10/2022
tweet - Jurgen Klopp wants Jude Bellingham but it will be tough for Liverpool to get him now. FSG simply don’t compete with the big spenders for players of this quality. It’s as simple as that
handles - 
keywords - LFC
media - https://t.co/gLQHtGixo
verified - True
---------------------------------
id - 1577780793270738947
locatio

,id,location,name,username,date,tweet,keywords,media,verified
0,1577793253197242368,Liverpool,Liverpool FC News,LivEchoLFC,2022-05-10,"""When they move away from home, it is time for...",LFC,https://t.co/greNJMInP,True
1,1577788221693509632,Liverpool,Liverpool FC News,LivEchoLFC,2022-05-10,Liverpool evening headlines,LFC,https://t.co/sHY3Zesvt,True
2,1577782825574694912,Dublin & London,Kevin Palmer,RealKevinPalmer,2022-05-10,Jurgen Klopp wants Jude Bellingham but it will...,LFC,https://t.co/gLQHtGixo,True
3,1577780793270738947,United States,AS USA,English_AS,2022-05-10,Darwin Núñez ⚡ Jurgen Klopp\n\n🇺🇾 The Uruguaya...,LFC,https://t.co/Je99oYCzU,True
4,1577780670448934937,None,HITC Sport,HITCsport,2022-05-10,Complete midfielder with huge experience\n\n | |,"LFC,THFC,Brozovic",,True
5,1577776895558557704,Liverpool,Liverpool FC News,LivEchoLFC,2022-05-10,Xabi Alonso must learn from mistakes of former...,LFC,https://t.co/greNK4jMp,True
6,1577773120726900744,Liverpool,Liverpool FC News,LivEchoLFC,2022-05-10,Liverpool will have no regrets about £11m tran...,LFC,https://t.co/DsvavEzwJ,True
7,1577770635836723201,None,Empire of the Kop,empireofthekop,2022-05-10,MORE ⬇️👇,LFC,https://t.co/a01AsGSSe,True
8,1577770633424998443,None,Empire of the Kop,empireofthekop,2022-05-10,Fabrizio Romano has urged everyone to 'get rea...,LFC,,True
9,1577769345899499550,Liverpool,Liverpool FC News,LivEchoLFC,2022-05-10,evening headlines,LFC,https://t.co/sHY3Zesvt,True


In [ ]:
sample_dict = {}
sample_dict['id'] = list(tweets_df['id'])
sample_dict['location'] = list(tweets_df['location'])
sample_dict['name'] = list(tweets_df['name'])

sample_df = pd.DataFrame(sample_dict)
display(sample_df)

,id,location,name
0,1577793253197242368,Liverpool,Liverpool FC News
1,1577788221693509632,Liverpool,Liverpool FC News
2,1577782825574694912,Dublin & London,Kevin Palmer
3,1577780793270738947,United States,AS USA
4,1577780670448934937,None,HITC Sport
5,1577776895558557704,Liverpool,Liverpool FC News
6,1577773120726900744,Liverpool,Liverpool FC News
7,1577770635836723201,None,Empire of the Kop
8,1577770633424998443,None,Empire of the Kop
9,1577769345899499550,Liverpool,Liverpool FC News


In [ ]:
import pyodbc
import sqlalchemy as sa

connection_string = (
    'Driver=ODBC Driver 17 for SQL Server;'
    'Server=KHURANSI01\SQLEXPRESS;'
    'Database=first_db;'
    'Trusted_Connection=no;'
    'UID=Sidharth;'
    'PWD=pwd;'
)
connection_url = sa.engine.URL.create(
    "mssql+pyodbc", 
    query={'odbc_connect':connection_string}
)
engine = sa.create_engine(connection_url, fast_executemany=True)

# Deleting existing data in SQL Table:-
with engine.begin() as conn:
    conn.exec_driver_sql("DELETE FROM dbo.twitter_data")

# upload the DataFrame
tweets_df.to_sql("twitter_data", engine, schema="dbo", if_exists="append", index=False)

DBAPIError: ignored